In [ ]:
import DataSetQuery
import datetime

#Sets the URL of the Malard ServiceGateway.
query = DataSetQuery.DataSetQuery('http://localhost:9000')

#Sets the output directory for all users of the server.
query.setEnvironment('test','/data/puma1/scratch/malard/export/','/data/puma1/scratch/malard/')
#Retrieves the environment that has just been set.
print(query.getEnvironment())

#determines the databases that store multiple datasets 
print(query.getParentDataSets())
#retrieve the data sets for a given parent
print(query.getDataSets('mtngla'))
#The overall data cube for the two datasets
print(query.getDataSetBoundingBox( 'mtngla','tandemx' ))
print(query.getDataSetBoundingBox( 'mtngla','srtm' ))

#Setup the bounding box
minX=0
maxX=100000
minY=0
maxY=100000
minT=datetime.datetime(2010,7,1,0,0)
maxT=datetime.datetime(2010,12,31,0,0)

#Queries the catalogue and returns the gridcells grouped by x and y with the min and max times.
gcs = query.getGridCells( 'mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT )
#print(gcs)

#Queries the catalogue to determine the shards that comprise the bounding box of interest.
shards = query.getShards('mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT)
#print(shards)



In [ ]:
#loads the json output of the getGridCells call and creates a flattened DataFrame.
from pandas.io.json import json_normalize
import json

data = json.loads(gcs)
df = json_normalize(data)

print(df)

totalPoints = df['totalPoints'].sum()
maxNumPoints = df['totalPoints'].max()

print('TP=[%d] MaxPoints in Cell=[%d]'%(totalPoints,maxNumPoints))   



In [ ]:
#Converts the Json output of the getShards call into a DataFrame.
dataShard = json.loads(shards)
dfShards = json_normalize(dataShard)

print(dfShards['shardName'])


In [ ]:
#Gets the available column names from the NetCDFs that are in the BoundingBox.
print(query.getDataSetColumns('mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT))

In [ ]:
import MalardHelpers

#Returns a file handle to a NetCDF that is a merge and filter of all the shards in a BoundingBox.
fileName = query.getNetCdfFile('mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT)

print("File created: %s" % (fileName))

#Converts the NetCDF into a DataFrame.
df = MalardHelpers.getDataFrameFromNetCDF(fileName)

print("Max Elevation %f" % (df['elev'].max()))
print("Min Elevation %f" % (df['elev'].min()))


In [ ]:

#a list of columns to see in the output (x,y,time will be added)
projections = ['elev','power','coh'] 
#a list of dictionaries that specify the filters
#only numeric fields are supported with the following operations gt(greater than), gte ( greater than equals), lt (less than) and lte (less than equals)
#filters are treated as and conditions
filters = [{'column':'power','op':'gt','threshold':10000},{'column':'coh','op':'gt','threshold':0.8}]

#No filters
nofilters = query.executeQuery( 'mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT,projections,[])
#Converts the NetCDF into a DataFrame.
dfNoFilter = MalardHelpers.getDataFrameFromNetCDF(nofilters)

print( "Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( dfNoFilter['coh'].max(),  dfNoFilter['coh'].min(), dfNoFilter['power'].max(), dfNoFilter['power'].min(), dfNoFilter['power'].count() ) )  

#With filters
withfilters = query.executeQuery( 'mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT,projections,filters)
#Converts the NetCDF into a DataFrame.
withFilter = MalardHelpers.getDataFrameFromNetCDF(withfilters)

print( "Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilter['coh'].max(),  withFilter['coh'].min(), withFilter['power'].max(), withFilter['power'].min(), withFilter['power'].count() ) )  

print( "Max Lon [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilter['x'].max(),  withFilter['coh'].min(), withFilter['power'].max(), withFilter['power'].min(), withFilter['power'].count() ) )  


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
## plot with no filters
plt.figure(figsize=(12,10))
plt.scatter(x=dfNoFilter['x'], y=dfNoFilter['y'], c=dfNoFilter['elev'], marker='.', s=20, vmax=7500)
plt.colorbar()
plt.show()

In [ ]:
## plot with filters
plt.figure(figsize=(12,10))
#vmin and vmax set min and max of colour bar
plt.scatter(x=withFilter['x'], y=withFilter['y'], c=withFilter['elev'], marker='.', s=20, vmax=dfNoFilter['elev'].max(), vmin=dfNoFilter['elev'].min())
plt.colorbar()
plt.show()

In [ ]:

#swathDetails - information about the source swath data and how the DataSetLoader did the gridding.

swathById = query.getSwathDetailsFromId('mtngla','tandemx',1)

print(swathById)
 
swathByName = query.getSwathDetailsFromName('mtngla','tandemx','CS_LTA__SIR_SIN_2S_20110713T134322_20110713T134453_C001.nc')

print(swathByName)


In [ ]:
#swathDetails - get all the swaths for a particular data set.
import pandas as pd
import MalardHelpers as m

df = m.getSwathDetailsAsDataFrame('mtngla','tandemx',query)

print(df.info())


In [ ]:
#sourcePath = #put your input path in here
#fileName = #put your test file in here
parentDsName = 'mtngla' 
maskType = 'Glacier' 
region = 'HMA' 
minX = 400000 
minY = 0 
size = 100000 

#query.publishMask(sourcePath, fileName, parentDsName, maskType, region, minX, minY, size )

In [ ]:
print(query.getMasks('mtngla'))

In [ ]:
parentdataset = 'mtngla'
maskType = 'Glacier'
region = 'HMA'

print(query.getGridCellMasks(parentdataset, maskType, region))

In [ ]:
#gets the mask for a gridcell.

print(query.getGridCellMask(parentdataset, maskType, region,minX,minY,size))